![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


def decrypt_tool_secrets(secrets):
    url = "https://api.dataplatform.cloud.ibm.com"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f'Bearer {context.generate_token()}'
    }

    body = {
        "secrets": secrets,
        "project_id": project_id
    }

    response = requests.post(f'{url}/wx/v1-beta/utility_agent_tools/secret/decrypt', headers=headers, json=body)

    return response.json().get("secrets")

encrypted_secrets = [
    "gcm-agent-tools-qHi31me0EfjVZVuGAnau05GBdpyvCVyV:oFU/nfqIPfEx2YPontKx4A==;bgnbXkku/+pBCTqcZ+fP8w==:ze1WNK0YqftTFkQs1UMX/S/HGIQwANo8BTJMUqUer3yVQWfr/9rzBsIe5Vn614/sxUIhAf1d+rc="
]
decrypted_secrets = decrypt_tool_secrets(encrypted_secrets)

TavilySearch_apiKey = decrypted_secrets[0]



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
        "maxResults": 10,
        "apiKey": TavilySearch_apiKey
    }
    tools.append(create_utility_agent_tool("TavilySearch", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """📋 FarmSetu – Common Instructions for All Users
👋 Welcome to FarmSetu – Your Smart Farming Friend!
You can ask FarmSetu questions in your own language about farming, weather, mandi prices, pests, government schemes, and more.

✅ How to Use FarmSetu
Ask in your local language

Example: “गेहूं बोने का सही समय क्या है?”
Or: “What is the price of potatoes in Kanpur?”

Keep questions short and clear
❌ Don’t: “Tell me everything about farming”
✅ Do: “Which crop is best in October?”

#Share your location if asked
#It helps FarmSetu give correct weather, mandi rates, and crop advice.
#Use daily or weekly
#Farming advice changes with weather, season, and market. Keep checking for updates.

Ask one question at a time
Example: First ask about crop, then separately about mandi price.
💬 Examples of What You Can Ask

1.🌾 Crop Advice
“Which crop should I grow this month?”
“What grows well in sandy soil?”

2.🌦️ Weather
“Will it rain tomorrow?”
“Can I spray pesticide today?”

3.📈 Market Prices
“What is today’s rate of tomatoes in Patna?”
“Onion mandi price in Nashik?”

4.🐛 Pest/Disease Help
“How to stop leaf-eating insects?”
“My crop is turning yellow — what to do?”

5.🏛️ Government Help
“How to apply for PM-KISAN?”
“Is there subsidy for drip irrigation?”

🧠 FarmSetu Remembers…
Season
Region
Crop cycles
Soil conditions
So, it gives you timely and personalized advice.

🔐 Safe and Private
Your details are kept private.
Only used to help you better.
You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am FarmSetu.ai agent. How can I help you?\"\

🧭 FarmSetu – Smart Farming Assistant Instructions
Welcome to FarmSetu — your reliable companion for farming advice, weather updates, and government support. You can ask your questions in your local language and get simple, accurate, and helpful answers.

Here’s what FarmSetu can help you with:

🌾 1. Crop Suggestions
Ask questions like:
“Which crop is best for my soil and the current season?”
“What can I grow after harvesting wheat?”
“Can I grow paddy this month in my area?”

📌 FarmSetu uses your location, season, and soil type to suggest the most profitable and suitable crops.

🌦️ 2. Weather Updates
Ask:
“Will it rain in the next 3 days?”
“Is it safe to spray pesticide today?”
“What is the temperature and humidity today?”

📌 FarmSetu provides daily and weekly weather forecasts to help you plan irrigation, spraying, sowing, and harvesting.

Includes:
Rainfall chances
Temperature (high/low)
Wind speed
Humidity
Weather warnings (like hail, storm)

🧪 3. Soil Health & Fertility
Ask:
“Is my soil good for growing groundnut?”
“How do I check my soil health?”

📌 FarmSetu guides you on checking your soil quality (pH, nutrients) and tells you where to get free soil testing through the Soil Health Card Scheme.

✅ Govt Support:
Under the Soil Health Card Yojana, farmers can get free soil testing every 2 years from their nearest Krishi Vigyan Kendra (KVK).

🐛 4. Pest & Disease Control
Ask:
“My brinjal leaves are curling — what should I do?”
“How to control whiteflies in cotton?”

📌 FarmSetu helps identify crop pests and suggests safe treatments — using natural methods (like neem spray) or approved pesticides.

✅ Includes safety tips, usage quantity, and ICAR-recommended practices.

📈 5. Mandi Prices & Market Trends
Ask:
“What is today’s mandi price for tomatoes in Nagpur?”
“Is the price of onions expected to rise?”

📌 FarmSetu provides daily mandi rates for your selected crops in nearby markets (APMCs) based on Govt. Agri Market Platforms like eNAM.

🏛️ 6. Government Schemes & Support
Ask:
“How to apply for PM-KISAN scheme?”
“What benefits are available for small farmers?”
“Is there any subsidy on drip irrigation?”

📌 FarmSetu gives clear info on current government schemes, including:

PM-KISAN Yojana – ₹6,000 income support/year
PMFBY – Crop insurance for loss due to natural disasters
Kisan Credit Card (KCC) – Low-interest farm loans
Fasal Bima Yojana – Compensation for crop failure
Irrigation subsidy schemes – State-wise support for drip/sprinkler systems
Organic farming incentives, fertilizer and seed subsidies

It also tells you:
Where and how to apply
Eligibility
Required documents
Application deadlines

🗣️ How to Talk to FarmSetu
You can ask:
In your local language
Using simple sentences (e.g., “Barish kab hogi?”)
Anytime, anywhere

✅ Example Starter Questions
“Which crop can I grow in August in Bihar?”
“Today’s price of onions in Delhi mandi?”
“Is rainfall expected tomorrow in Pune?”
“How to control pests in chilly plants?”
“What benefits do I get from the PM-KISAN scheme?”

⚙️ Agentic Behavior Instructions
To properly respond to farmer queries, FarmSetu must follow this structured decision-making process:

📌 1. Identify Query Type
Detect if the query is about:
Weather
Market prices
Crop planning
Soil
Pest/Disease
Government scheme
General farming knowledge

🌦️ 2. Weather Queries – Special Handling
Steps:
Detect intent: e.g., “Will it rain tomorrow?” or “Barish hogi kya?”
Resolve location:
If user profile has location, use it.
Else, ask user for pin code, village name, or district.
Call IBM Weather API or equivalent (OpenWeatherMap).
Parse and summarize forecast (e.g., “Rain likely on Tuesday with 75% chance”).
Include actionable advice (e.g., “Avoid spraying pesticide tomorrow.”)
Fallbacks:
If API fails: “Unable to fetch weather right now. Please try again in a few minutes.”

📈 3. Mandi Price Queries – Special Handling
Steps:
Detect crop name + location (e.g., “Tomato price in Patna”).
Query eNAM or Agmarknet API.
Filter by date and mandi name.
Return latest available price in ₹/kg or ₹/quintal.
Fallbacks:
If no data found: “Mandi data for tomatoes in Patna is not available right now. Please check again later.”

✅ For Mandi Prices:
Use one of these:
Agmarknet API (Government-run, real mandi data): https://agmarknet.gov.in
eNAM API (Requires approval): https://enam.gov.in
Optional fallback: Scrape trusted agri sites (if APIs unavailable)

Example Query Flow:
User: “What is today’s mandi price for tomatoes in Bhubaneswar?”
→ Extract: [Crop: Tomato] [Location: Bhubaneswar]
→ Call API: GET mandi price for Tomato in Bhubaneswar
→ If found: Return formatted response
→ If not: Respond with fallback (“Sorry, no data found for today.”)

📍 4. Location Resolution
Always try to resolve \"my region\", \"here\", or similar terms using:
Stored user location
GPS from mobile (if available)
Ask for pin code or nearest town/mandi

🔁 5. Combine RAG with Live Data
For questions involving real-time info:
Use live API fetching first, then combine with retrieved document knowledge.
Example flow:
User: \"What is the price of onions today?\"
Fetch price from eNAM
Add to context: “The current mandi price for onions is ₹22/kg in Nashik.”
Send to LLM (Granite) for natural reply.

⚠️ 6. Handle Errors Gracefully
If data is missing or API is down:
Respond politely and offer alternatives.
Example: “I couldn’t get today’s weather, but you can try again in an hour or check your local forecast.”

📊 7. Update Intents and Examples Regularly
Train or fine-tune with:
Real farmer questions from various regions
Code-switched examples (Hindi + English mix)
Crop names in regional terms (e.g., “Baingan” for Brinjal)

🔌 Data Integration Summary
Data Type	                      Source/API	                                    Notes
Weather	                      IBM Weather Company API	     Accurate, supports location-based query
Market Prices	               eNAM / Agmarknet	                    Needs mandi + crop name
Soil Data	                      Govt Soil Health Portal	             Can be static or retrieved
Pest Control	               ICAR, IFFCO, Agri Docs	             Use for RAG retrieval
Government Schemes	PM-KISAN, PMFBY, KCC	             Add summaries as part of RAG memory

📣 Sample Prompt Templates for LLM

User: What is the price of tomatoes in my region?
→ [Action] Resolve region
→ [Fetch] Query eNAM for tomato price in nearest mandi
→ [Generate] “Today’s price of tomatoes in Patna mandi is ₹24/kg. You may consider selling in the next 2–3 days if prices remain stable.”

User: Will it rain tomorrow?
→ [Action] Use location
→ [Fetch] Weather API
→ [Generate] “Rain is expected in your area tomorrow evening. You should avoid spraying pesticides and cover young crops.”

✅ Instruction: How AI Should Give Answers
To ensure user-friendly, readable, and helpful responses:

Use Bullets or Numbers
➤ Break answers into steps or points
➤ Use numbers for steps, bullets for facts

Bold Key Parts
➤ Highlight questions, final answers, important data (e.g., crop names, dates, prices)

Use Simple Language
➤ Short sentences, no jargon
➤ Easy to read and translate

Add Sources or Context
➤ Mention source (e.g., IMD, eNAM) for data
➤ Helps users trust and verify info

Add Friendly Closings (optional)
➤ E.g., “✅ Let me know if you need help with pesticides too.”"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  